In [ ]:
###从https://www.iplant.cn/中检索图像并完成图像下载
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd

if __name__ == '__main__':
    inpath = r"H:\植物智图像下载\未分类"
    table = pd.read_excel(r"H:\植物智图像下载\未分类\植物信息汇总表.xlsx")
    dowedtable = pd.read_excel(r"H:\植物智图像下载\未分类\已下载分类信息表.xlsx")
    downloadf = pd.read_excel(r"H:\植物智图像下载\未分类\已下载.xlsx")
    downloadedname = dowedtable["name"].tolist()
    sorttable = table.loc[(table["ImageCount"] < 500) & (~table["Classname"].isin(downloadedname))]
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.maximize_window()
    link_dict = dict()
    for i,item in enumerate(sorttable["Classname"]):
        url = f"https://www.iplant.cn/info/{item}?t=p"
        driver.get(url)
        sleep(1)
        link_dict[item] = list()
        #获取输入框位置，进行信息植物信息提交
        try:
            while True:
                driver.find_element(By.LINK_TEXT,'下一页').click()
                sleep(0.3)
                images_counts = driver.find_elements(By.CLASS_NAME,'img')
                for imgdiv in images_counts:
                    image_pid = imgdiv.get_attribute("pid")
                    image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"
                    print(image_url)
                    link_dict[item].append(image_url)
        except selenium.common.NoSuchElementException as e:
            print(e)

In [ ]:
import pickle
f = open(r"H:\植物智图像下载\未分类\plant_url_nodown.pkl","wb")
pickle.dump(link_dict,f)
f.close()

In [101]:
##通过selenium模拟浏览器批量下载图像数据
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
from selenium.webdriver.chrome.options import Options
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd
import pyautogui
import time
import pickle
import random

if __name__ == '__main__':
    fd = open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\All_link.pkl","rb")
    link_dict = pickle.load(fd)
    fd.close()
    outpath = r"C:\Users\DELL\Downloads"
    #downloadpath = r"I:\植物智图像下载\分类"
    #######-----------------------------------------------------------------设置代理IP----------------------------------------------------------
    #ipf = r"E:\PlantClassficationDetect\PlantDetecter\ip.txt"
    #with open(ipf,"r") as fid:
    #    ips = fid.readlines()
    proxy_arr = [
        '--proxy-server=http://124.70.55.29:20002',
 ]
    Chrome_options = Options()

    proxy = random.choice(proxy_arr)         #随机选择一个代理
    Chrome_options.add_argument(proxy)
    #driver = webdriver.Chrome(options=Chrome_options)
    driver = webdriver.Chrome()
    driver.implicitly_wait(2)
    driver.maximize_window()
    ###########-------------------------------------------------------------模拟登陆-----------------------------------------------------------------------------------
    logurl = "https://ppbc.iplant.cn/login"
    #downloadf = pd.read_excel(r"I:\植物智图像下载\未分类\已下载.xlsx")
    #plantfname = downloadf["name"].tolist()
    #模拟登录
    #uid = "王永财"
    #id = "王辰阳"
    #uid = "赵一曼"
    #uid = "王诚毅"
    uid = "李健"
    #uid = "王佳芝"
    #uid = "骆驼祥子"
    #uid = "孔乙已"
    #uid = "祥林嫂"
    #uid = "闰土"
    #pwd = "KBqinfang914361"
    pwd = "abc@123456"
    driver.get(logurl)
    driver.find_element(By.ID,"userInput").send_keys(uid)
    driver.find_element(By.ID,"userPwd").send_keys(pwd)
    driver.find_element(By.ID,"btnlog").click()
    ######--------------------------------------------------------------开始下载图像-----------------------------------------------------------------------------
    time.sleep(1)
    for k,value in list(link_dict.items()):     #向队列中放入数据
        #downloaded_class = os.path.join(downloadpath,k)
        if k == "樟子松" or k == "花楸树" or k == "辽宁山楂" or k == "山里红" or k == "山荆子" or k == "接骨木" or k == "菊花":    #设置跳过的类别
            continue
        #判断每个类别已下载图像个数是否达到要求，如果达不到则继续下载，否则则调到一个类别
        noready_f = [l for l in value if not os.path.exists(os.path.join(outpath,os.path.basename(l)))]
        noready_count = len(noready_f)
        ready_count = len(value) - noready_count     #已经下载的图像个数
        if len(value) > 1000 and ready_count < 1000:
            urllist = random.sample(noready_f,1000 - ready_count)      #如果下载条数大于500，随机选择500个即可
            for url in urllist:
                imageid = os.path.basename(url).split(".")[0]
                outfname = os.path.join(outpath,os.path.basename(url))
                #downloadfname = os.path.join(downloaded_class,os.path.basename(url))
                imagepageurl = f"https://ppbc.iplant.cn/tu/{imageid}"
                #if os.path.basename(url) in plantfname:
                #    continue
                if os.path.exists(outfname) and os.path.getsize(outfname) > 12040:
                    continue
                #elif os.path.exists(downloadfname):
                #    continue
                elif os.path.exists(outfname) and os.path.getsize(outfname) < 12040:
                    try:
                        driver.get(imagepageurl)
                        pic = driver.find_element(By.XPATH,'//*[@id="viewer2"]/img')
                        action = ActionChains(driver).move_to_element(pic)  # 移动到该元素
                        action.context_click(pic)  # 右键点击该元素
                        action.perform() # 执行
                        pyautogui.typewrite(['v']) # 敲击V进行保存
                        time.sleep(0.5)
                        pyautogui.typewrite(['enter'])
                    except Exception as e:
                        print(e)
                else:
                    try:
                        driver.get(imagepageurl)
                        pic = driver.find_element(By.XPATH,'//*[@id="viewer2"]/img')
                        action = ActionChains(driver).move_to_element(pic)  # 移动到该元素
                        action.context_click(pic)  # 右键点击该元素
                        action.perform() # 执行
                        pyautogui.typewrite(['v']) # 敲击V进行保存
                        time.sleep(0.5)
                        pyautogui.typewrite(['enter'])
                    except Exception as e:
                        print(e)

In [35]:
##通过selenium模拟浏览器批量下载图像数据
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
from selenium.webdriver.chrome.options import Options
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd
import pyautogui
import time
import pickle
import random

if __name__ == '__main__':
    fd = open(r"C:\Users\wangyc\Downloads\FaildDownload.pkl","rb")
    link_dict = pickle.load(fd)
    fd.close()
    outpath = r"C:\Users\wangyc\Downloads"
    #downloadpath = r"I:\植物智图像下载\分类"
    #######-----------------------------------------------------------------设置代理IP----------------------------------------------------------
    #ipf = r"E:\PlantClassficationDetect\PlantDetecter\ip.txt"
    #with open(ipf,"r") as fid:
    #    ips = fid.readlines()
    proxy_arr = [
        '--proxy-server=http://124.70.55.29:20002',
 ]
    #Chrome_options = Options()

    #proxy = random.choice(proxy_arr)         #随机选择一个代理
    #Chrome_options.add_argument(proxy)
    #driver = webdriver.Chrome(options=Chrome_options)
    driver = webdriver.Chrome()
    driver.implicitly_wait(2)
    driver.maximize_window()
    ###########-----------------------模拟登陆-----------------------------
    logurl = "https://ppbc.iplant.cn/login"
    #downloadf = pd.read_excel(r"I:\植物智图像下载\未分类\已下载.xlsx")
    #plantfname = downloadf["name"].tolist()
    #模拟登录
    uid = "王永财"
    #uid = "王辰阳"
    #uid = "赵一曼"
    #uid = "王诚毅"
    #uid = "李健"
    #uid = "王佳芝"
    #uid = "骆驼祥子"
    #uid = "孔乙已"
    #uid = "祥林嫂"
    #uid = "闰土"
    #uid = "阿Q"
    pwd = "KBqinfang914361"
    #pwd = "abc@123456"
    driver.get(logurl)
    driver.find_element(By.ID,"userInput").send_keys(uid)
    driver.find_element(By.ID,"userPwd").send_keys(pwd)
    driver.find_element(By.ID,"btnlog").click()
    ######--------------------------------------------------------------开始下载图像-----------------------------------------------------------------------------
    time.sleep(1)
    for k,value in list(link_dict.items())[-65:-60]:     #向队列中放入数据
        #downloaded_class = os.path.join(downloadpath,k)
        if k == "樟子松" or k == "花楸树" or k == "辽宁山楂" or k == "山里红" or k == "山荆子" or k == "接骨木" or k == "菊花":    #设置跳过的类别
            continue
        #判断每个类别已下载图像个数是否达到要求，如果达不到则继续下载，否则则调到一个类别
        noready_f = [l for l in value if not os.path.exists(os.path.join(outpath,os.path.basename(l)))]
        noready_count = len(noready_f)
        ready_count = len(value) - noready_count     #已经下载的图像个数
        if len(value) > 500 and ready_count < 500:
            urllist = random.sample(noready_f,500 - ready_count)      #如果下载条数大于500，随机选择500个即可
            for url in urllist:
                imageid = os.path.basename(url).split(".")[0]
                outfname = os.path.join(outpath,os.path.basename(url))
                #downloadfname = os.path.join(downloaded_class,os.path.basename(url))
                imagepageurl = f"https://ppbc.iplant.cn/tu/{imageid}"
                #if os.path.basename(url) in plantfname:
                #    continue
                if os.path.exists(outfname) and os.path.getsize(outfname) > 12040:
                    continue
                #elif os.path.exists(downloadfname):
                #    continue
                elif os.path.exists(outfname) and os.path.getsize(outfname) < 12040:
                    try:
                        driver.get(imagepageurl)
                        pic = driver.find_element(By.XPATH,'//*[@id="viewer2"]/img')
                        action = ActionChains(driver).move_to_element(pic)  # 移动到该元素
                        action.context_click(pic)  # 右键点击该元素
                        action.perform() # 执行
                        pyautogui.typewrite(['v']) # 敲击V进行保存
                        time.sleep(0.5)
                        pyautogui.typewrite(['enter'])
                    except Exception as e:
                        print(e)
                else:
                    try:
                        driver.get(imagepageurl)
                        pic = driver.find_element(By.XPATH,'//*[@id="viewer2"]/img')
                        action = ActionChains(driver).move_to_element(pic)  # 移动到该元素
                        action.context_click(pic)  # 右键点击该元素
                        action.perform() # 执行
                        pyautogui.typewrite(['v']) # 敲击V进行保存
                        time.sleep(0.5)
                        pyautogui.typewrite(['enter'])
                    except Exception as e:
                        print(e)

Alert Text: 今日实名用户访问次数已超
Message: unexpected alert open: {Alert text : 今日实名用户访问次数已超}
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x0107DCE3]
	(No symbol) [0x010139D1]
	(No symbol) [0x00F24DA8]
	(No symbol) [0x00F7D9C8]
	(No symbol) [0x00F6ACC6]
	(No symbol) [0x00F46F68]
	(No symbol) [0x00F480CD]
	GetHandleVerifier [0x012F3832+2506274]
	GetHandleVerifier [0x01329794+2727300]
	GetHandleVerifier [0x0132E36C+2746716]
	GetHandleVerifier [0x01126690+617600]
	(No symbol) [0x0101C712]
	(No symbol) [0x01021FF8]
	(No symbol) [0x010220DB]
	(No symbol) [0x0102C63B]
	BaseThreadInitThunk [0x773A00F9+25]
	RtlGetAppContainerNamedObjectPath [0x774E7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x774E7B8E+238]

Alert Text: 今日实名用户访问次数已超
Message: unexpected alert open: {Alert text : 今日实名用户访问次数已超}
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x0107DCE3]
	(No symbol) [0x010139D1]
	(No symbol) [0x00F24DA8]
	(No symbol) [0x00F7D9C8]
	(No symbol) 

In [17]:

###从https://www.iplant.cn/中检索图像并完成图像下载，避免第一页被跳过
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import pickle

def isElement(by,value):
    try:
        element = driver.find_element(by=by,value=value)
    except NoSuchElementException as e:
        print(e)
        return True
    else:
        return False
if __name__ == '__main__':
    table = pd.read_excel(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\重新梳理物种.xlsx")
    table = table["名称"]
    #table = ["阴山乌头","阿拉善点地梅","阿拉善黄芪","乌拉特黄芪","短叶假木贼","草苁蓉","兴安柴胡","兴安柴胡","阿拉善沙拐枣","蒙古旱雀豆",
    #         "盐生肉苁蓉","斑子麻黄","圆果甘草","裸果木","针枝芸香","羊柴","蒙古羊柴","戈壁藜","长梗扁桃","斧翅沙芥",
    #         "沙芥","黄花红砂","阿拉善风毛菊","内蒙西风芹","大花雀儿豆","百花蒿","短瓣金莲花","三叶犁头尖"]
    #table = ["黄花鸢尾","马蔺","曙南芥","桃叶鸦葱"]
    #sorttable = table.loc[table["ImageCount"] < 500]
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.maximize_window()
    link_dict = dict()
    for i,item in enumerate(table):
        url = f"https://www.iplant.cn/info/{item}?t=p"
        driver.get(url)
        sleep(0.5)
        link_dict[item] = list()
        #获取输入框位置，进行信息植物信息提交
        sleep(1)
        res = isElement(By.LINK_TEXT,"下一页")
        if res is True:
            images_counts = driver.find_elements(By.CLASS_NAME,'img')
            for imgdiv in images_counts:
                image_pid = imgdiv.get_attribute("pid")
                image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"
                print(image_url)
                link_dict[item].append(image_url)
        else:
            try:
                images_counts = driver.find_elements(By.CLASS_NAME,'img')
                for imgdiv in images_counts:
                    image_pid = imgdiv.get_attribute("pid")
                    image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"
                    print(image_url)
                    link_dict[item].append(image_url)
                while True:
                    driver.find_element(By.LINK_TEXT,'下一页').click()
                    images_counts = driver.find_elements(By.CLASS_NAME,'img')
                    sleep(0.5)
                    for imgdiv in images_counts:
                        image_pid = imgdiv.get_attribute("pid")
                        image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"
                        print(image_url)
                        link_dict[item].append(image_url)
            except selenium.common.NoSuchElementException as e:
                print(e)
    with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\FaildDownload.pkl","wb") as fd:
        pickle.dump(link_dict,fd)

PermissionError: [Errno 13] Permission denied: 'E:\\PlantClassficationDetect\\植物分类\\呼伦贝尔植物名录数据\\重新梳理物种.xlsx'

In [ ]:
###获取每个类别每个图像采集信息等重要信息
import os
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from urllib.error import URLError
from time import sleep
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import pickle
import glob
import pandas as pd

def isElement(by,value):
    try:
        element = driver.find_element(by=by,value=value)
    except NoSuchElementException as e:
        print(e)
        return True
    else:
        return False
if __name__ == '__main__':
    #table = pd.read_excel(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\重新梳理物种.xlsx")
    #table = table["名称"]
    inpath = r"I:\植物分类\呼伦贝尔植物名录数据"
    plantfolder = [f for f in os.listdir(inpath) if os.path.isdir(os.path.join(inpath,f))]
    plantcont = [len(glob.glob(os.path.join(inpath,f,"*.jpg"))) for f in plantfolder]
    plantname = [plantfolder[i] for i,count in enumerate(plantcont) if count > 0]
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.maximize_window()
    link_dict = pd.DataFrame(columns=["ChineseName","Latin","ID","Organ","Author","Location","ImageUrl"])
    for i,item in enumerate(table):
        url = f"https://www.iplant.cn/info/{item}?t=p"
        driver.get(url)
        sleep(0.5)
        link_dict[item] = list()
        sleep(1)
        res = isElement(By.LINK_TEXT,"下一页")
        if res is True:
            images_counts = driver.find_elements(By.CLASS_NAME,'img')
            title_counts = driver.find_elements(By.CLASS_NAME,'title')
            for imgdiv in images_counts:
                image_pid = imgdiv.get_attribute("pid")
                image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"

                print(image_url)
                link_dict[item].append(image_url)
        else:
            try:
                images_counts = driver.find_elements(By.CLASS_NAME,'img')
                for imgdiv in images_counts:
                    image_pid = imgdiv.get_attribute("pid")
                    image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"
                    print(image_url)
                    link_dict[item].append(image_url)
                while True:
                    driver.find_element(By.LINK_TEXT,'下一页').click()
                    images_counts = driver.find_elements(By.CLASS_NAME,'img')
                    sleep(0.5)
                    for imgdiv in images_counts:
                        image_pid = imgdiv.get_attribute("pid")
                        image_url = f"https://img6.iplant.cn/image61/b/{image_pid}.jpg"
                        print(image_url)
                        link_dict[item].append(image_url)
            except selenium.common.NoSuchElementException as e:
                print(e)
    with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\FaildDownload.pkl","wb") as fd:
        pickle.dump(link_dict,fd)

In [129]:
import pickle
fd = open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\FaildDownload.pkl","rb")
link_dict = pickle.load(fd)
fd.close()
i = 0
for key,items in link_dict.items():
    i += len(items)
print(i)

74602


In [18]:
import os
import pickle
with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\664-762.pkl","wb") as fd:
    pickle.dump(link_dict,fd)

In [24]:
import os
import pickle

with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\林向荆-杉叶藻.pkl","rb") as fd1:
        link_dict1 = pickle.load(fd1)
with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\341-473.pkl","rb") as fd2:
        link_dict2 = pickle.load(fd2)
with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\474-520.pkl","rb") as fd3:
        link_dict3 = pickle.load(fd3)
with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\521-663.pkl","rb") as fd4:
        link_dict4 = pickle.load(fd4)
with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\664-762.pkl","rb") as fd5:
        link_dict5 = pickle.load(fd5)
with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\762-.pkl","rb") as fd6:
        link_dict6 = pickle.load(fd6)
link_dict = dict()
link_dict.update(link_dict1)
link_dict.update(link_dict2)
link_dict.update(link_dict3)
link_dict.update(link_dict4)
link_dict.update(link_dict5)
link_dict.update(link_dict6)
print(len(link_dict.keys()))
i = 0
for key,items in link_dict.items():
    i += len(items)
print(i)
#with open(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\All_link.pkl","wb") as fdall:
#    pickle.dump(link_dict,fdall)

792
380903


In [1]:
import os
import shutil
import pickle
if __name__ == '__main__':
    inpath = r"C:\Users\wangyc\Downloads"
    outpath = r"H:\植物智图像下载\temp"
    fd = open(r"C:\Users\wangyc\Downloads\FaildDownload.pkl","rb")
    link_dict = pickle.load(fd)
    fd.close()
    for k,value in link_dict.items():
        classfolder = os.path.join(outpath,k)
        if not os.path.exists(classfolder):
            os.makedirs(classfolder)
        for url in value:
            img = os.path.basename(url)
            imgname = os.path.join(inpath,img)
            if os.path.isfile(imgname) and os.path.exists(imgname) and not os.path.exists(os.path.join(classfolder,img)):
                shutil.copy(imgname,classfolder)
                print(imgname)

C:\Users\wangyc\Downloads\2526409.jpg
C:\Users\wangyc\Downloads\1490779.jpg
C:\Users\wangyc\Downloads\15562.jpg
C:\Users\wangyc\Downloads\658457.jpg
C:\Users\wangyc\Downloads\6172575.jpg
C:\Users\wangyc\Downloads\6172180.jpg
C:\Users\wangyc\Downloads\6227208.jpg
C:\Users\wangyc\Downloads\6122737.jpg
C:\Users\wangyc\Downloads\6091826.jpg
C:\Users\wangyc\Downloads\6091824.jpg
C:\Users\wangyc\Downloads\6086783.jpg
C:\Users\wangyc\Downloads\6040721.jpg
C:\Users\wangyc\Downloads\5995741.jpg
C:\Users\wangyc\Downloads\5992058.jpg
C:\Users\wangyc\Downloads\5934358.jpg
C:\Users\wangyc\Downloads\5934356.jpg
C:\Users\wangyc\Downloads\5916135.jpg
C:\Users\wangyc\Downloads\5769042.jpg
C:\Users\wangyc\Downloads\5769031.jpg
C:\Users\wangyc\Downloads\5734016.jpg
C:\Users\wangyc\Downloads\5703622.jpg
C:\Users\wangyc\Downloads\5703621.jpg
C:\Users\wangyc\Downloads\5588363.jpg
C:\Users\wangyc\Downloads\5483638.jpg
C:\Users\wangyc\Downloads\5452289.jpg
C:\Users\wangyc\Downloads\5257859.jpg
C:\Users\wangyc

In [38]:
0#统计每个种类的图像个数
import os
import pandas as pd
import glob
if __name__ == '__main__':
    inpath = r"I:\植物分类\植物图像库合并"
    df = pd.DataFrame(columns=["名称","图像数量"])
    for i,name in enumerate(os.listdir(inpath)):
        subf = os.path.join(inpath,name)
        if os.path.isdir(subf):
            flist = glob.glob(os.path.join(subf,"*"))
            count = len(flist)
            df.loc[i,"名称"] = name
            df.loc[i,"图像数量"] = count
    with pd.ExcelWriter(os.path.join(inpath,"合并图像库各类图像数量.xlsx")) as writer:
        df.to_excel(writer)

In [9]:
####将多个名录进行合并操作，保持唯一
######将两个不同的植物名录
import os
import pandas as pd

if __name__ == '__main__':
    hulun = r"E:\PlauntClassficationDetect\植物分类\呼伦贝尔植物名录数据\呼伦贝尔植物名录.xlsx"      #呼伦贝尔植物名录总共1000种
    #xilin = r"E:\PlantClassficationDetect\植物分类\三大样地植物库筛查\锡林浩特样地物种调查名录.xlsx"
    #eds = r"E:\PlantClassficationDetect\植物分类\三大样地植物库筛查\鄂尔多斯样地物种调查名录.xlsx"
    all = r"E:\PlantClassficationDetect\植物分类\植物图像库合并\合并图像库各类图像数量.xlsx"    #已下载图像总共400种
    dbf_hulun = pd.read_excel(hulun)
    #dbf_xilin = pd.read_excel(xilin)
    #dbf_eds = pd.read_excel(eds)
    dbf_all = pd.read_excel(all)

    allclass = dbf_all["名称"].tolist()
    class_hulun = dbf_hulun["种名"].tolist()
    #class_xilin = dbf_xilin["中文名"].tolist()
    #class_eds = dbf_eds["中文名"].tolist()
    #classname = set(class_hulun + class_xilin + class_eds)
    #df = pd.DataFrame(columns=["中文名"])
    #df["中文名"] = list(classname)
    #with pd.ExcelWriter(r"E:\PlantClassficationDetect\植物分类\三大样地植物库筛查\三个样地植物名录合并.xlsx") as writer:
    #    df.to_excel(writer)
    #class_rest = list()
    df_rest = pd.DataFrame(columns=dbf_hulun.columns)
    for i,name in enumerate(class_hulun):
        if name not in allclass:
            df_rest.loc[i,:] = dbf_hulun.loc[i,:]
    with pd.ExcelWriter(r"E:\PlantClassficationDetect\植物分类\呼伦贝尔植物名录数据\已下载不在名录.xlsx") as writer:
        df_rest.to_excel(writer)